# Lesson 1: The Python language (workbook)

Import statements: run these first.

In [ ]:
# Python standard library
import numbers
import json
import math

# Scientific Python ecosystem
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Particle physics tools
from hepunits import GeV
from particle import Particle
import vector

Use these empty cells to try things on your own.

We'll be using these equations a lot:

$$p = \sqrt{{p_x}^2 + {p_y}^2 + {p_z}^2}$$

$$m = \sqrt{E^2 - p^2}$$

**Quizlet:** Fix the mistake!

In [ ]:
E = 68.1289790
px = -17.945541
py = 13.1652603
pz = 64.3908386

In [ ]:
m = (E**2 - px**2 + py**2 + pz**2)**(1/2)
m

`m` should be `0.10565709514578395`. What was the mistake?

What can you do with this `muon` object?

In [ ]:
muon = Particle.from_name("mu+")
muon

In [ ]:
?muon

In [ ]:
dir(muon)

**Quizlet:** before you run the following, what will it do?

```python
type(type(1)("2"))
```

**Quizlet:** Before you run it, what will this do?

```python
some_list[2:8][3]
```

In [ ]:
some_list = [0.0, 1.1, 2.2, 33333, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9, 'mixed types']
some_dict = {'one': 1.1, 'two': 22222, 'three': 3.3, 123: 'mixed types'}

Starting from the observed electrons and muons, we reconstruct unobserved particles by adding energy and momentum.

<br>

<center>
<img src="../img/higgs-to-four-leptons-diagram.png" width="50%">
</center>

In [ ]:
particles = [
    {"type": "electron", "E": 171.848714, "px": 38.4242935, "py": -28.779644, "pz": 165.006927, "charge": 1,},
    {"type": "electron", "E": 138.501266, "px": -34.431419, "py": 24.6730384, "pz": 131.864776, "charge": -1,},
    {"type": "muon", "E": 68.1289790, "px": -17.945541, "py": 13.1652603, "pz": 64.3908386, "charge": 1,},
    {"type": "muon", "E": 18.8320473, "px": -8.1843795, "py": -7.6400470, "pz": 15.1420097, "charge": -1,},
]

In [ ]:
def particle_decay(name, particle1, particle2):
    return {
        "type": name,
        "E": particle1["E"] + particle2["E"],
        "px": particle1["px"] + particle2["px"],
        "py": particle1["py"] + particle2["py"],
        "pz": particle1["pz"] + particle2["pz"],
        "charge": particle1["charge"] + particle2["charge"],
    }

In [ ]:
z1 = particle_decay("Z boson", particles[0], particles[1])
z1

In [ ]:
z2 = particle_decay("Z boson", particles[2], particles[3])
z2

In [ ]:
higgs = particle_decay("Higgs boson", z1, z2)
higgs

**Quizlet:** Define the `particle_mass` function and compute the mass of `z1`, `z2`, and `higgs`.

| input | expected output |
|:--|:--|
| `particle_mass(z1)` | `90.28562890933117` |
| `particle_mass(z2)` | `22.878929369436445` |
| `particle_mass(higgs)` | `125.23413366311769` |

In [ ]:
def particle_mass(particle):
    ...

### Mini-project: let's make an event display

In [ ]:
dataset = json.load(open("../data/SMHiggsToZZTo4L.json"))

In [ ]:
def to_vector(particle):
    return vector.obj(
        pt=particle["pt"],
        eta=particle["eta"],
        phi=particle["phi"],
        mass=particle["mass"],
    )

In [ ]:
def beamline(ax):
    ax.plot([0, 0], [0, 0], [-100, 100], c="black", ls=":")

In [ ]:
def cms_outline(ax):
    z = np.linspace(-100, 100, 50)
    theta = np.linspace(0, 2 * np.pi, 12)
    theta_grid, z_grid = np.meshgrid(theta, z)
    x_grid = 100 * np.cos(theta_grid)
    y_grid = 100 * np.sin(theta_grid)
    ax.plot_surface(x_grid, y_grid, z_grid, alpha=0.2, color="red")

In [ ]:
class Particle:
    def __init__(self, E, px, py, pz):
        self.E = E
        self.px = px
        self.py = py
        self.pz = pz

    def __repr__(self):
        return (
            f"<{type(self).__name__} E={self.E} px={self.px} py={self.py} pz={self.pz}>"
        )

    def draw(self, ax):
        raise NotImplementedError(f"{type(self).__name__} has not been implemented yet")

In [ ]:
class Electron(Particle):
    def draw(self, ax):
        ax.plot([0, self.px], [0, self.py], [0, self.pz], c="blue")

In [ ]:
class Muon(Particle):
    def draw(self, ax):
        ax.plot([0, self.px], [0, self.py], [0, self.pz], c="green")

In [ ]:
%matplotlib widget

fig = plt.figure()

In [ ]:
event = dataset[96]  # a nice event with 3 electrons and 3 muons

electron_objects = []
for particle in event["electron"]:
    v = to_vector(particle)
    electron_objects.append(Electron(v.E, v.px, v.py, v.pz))

muon_objects = []
for particle in event["muon"]:
    v = to_vector(particle)
    muon_objects.append(Muon(v.E, v.px, v.py, v.pz))

fig.clf()
ax = fig.add_subplot(111, projection="3d")

beamline(ax)
cms_outline(ax)

for electron in electron_objects:
    electron.draw(ax)

for muon in muon_objects:
    muon.draw(ax)